## 05. PyTorch Going Modular

#### => Turn useful notebook code cells into reusable Python scripts (.py files)
>
> e.g.: 
> - `data_setup.py` -> a file to prepare and download data if needed
> - `engine.py` -> a file containing various training functions
> - `model_builder.py` or `model.py` -> a file to create a PyTorch model
> - `train.py` -> a file to leverage all other files and train a target PyTorch model
> - `utils.py` -> a file dedicated to helpful utility functions

#### => Why go modular?
>
>  larger scale projects -> Python scripts more reproducible and easier to run.

#### How do I turn my notebook code into Python scripts?

**`Script mode:` uses `Jupyter Notebook cell magic` (special commands) to turn specific cells into Python scripts.**

Putting `%%writefile filename` at the top of a cell in Jupyter or Google Colab will write the contents of that cell to a specified filename.
> => e.g. 
>
> %%writefile hello_world.py
>
> print("hello world, machine learning is fun!")


### 0. Create a folder for storing Python scripts

In [1]:
import os
os.makedirs("go_modular", exist_ok=True)

### 1. Get Data

In [2]:
import os
import zipfile
from pathlib import Path
import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

data/pizza_steak_sushi directory exists.


In [3]:
# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Unzipping pizza, steak, sushi data...


In [4]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

### 2. Create Datasets and DataLoaders (script mode)

=> use the Jupyter magic function to create a `.py` file for creating DataLoaders.

=> save a code cell's contents to a file using the Jupyter magic `%%writefile filename`

In [5]:
%%writefile go_modular/data_setup.py

"""
contains functionality for creating PyTirch DataLoader's for image
classification data
"""

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size: int):
    # Use ImageFolder to create datasets(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes
    
    # Turn images into DataLoaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True 
    )

    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True 
    )

    return train_dataloader, test_dataloader, class_names

Writing go_modular/data_setup.py


In [6]:
from torchvision import transforms

# Create simple transform
data_transform = transforms.Compose([ 
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [7]:
from go_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x155cbacb0>,
 ['pizza', 'steak', 'sushi'])

### 3. Make a model with a script

In [8]:
%%writefile go_modular/model_builder.py

import torch
from torch import nn

class TinyVGG(nn.Module):
    """ model architecture copying TingVGG from CNN Explainer """
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
        self.con_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.con_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 15 * 15, out_features=output_shape)
        )
    
    def forward(self, x):
        x = self.con_block_1(x)
        x = self.con_block_2(x)
        x = self.classifier(x)
        return x

Writing go_modular/model_builder.py


In [9]:
import torch
from go_modular import model_builder

torch.manual_seed(42)

model_0 = model_builder.TinyVGG(input_shape=3,
                                hidden_units=10,
                                output_shape=len(class_names))
model_0

TinyVGG(
  (con_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (con_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2250, out_features=3, bias=True)
  )
)

In [10]:
# try a forward pass
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single)
    
# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0264, -0.0172, -0.0141]])

Output prediction probabilities:
tensor([[0.3427, 0.3281, 0.3291]])

Output prediction label:
tensor([0])

Actual label:
1


### 4. Turn training functions into a script

In [11]:
%%writefile go_modular/engine.py

""" contains functions for training and testing a PyTorch model """

from typing import Dict, List, Tuple
import torch
from torch import nn
from tqdm.auto import tqdm

# training steps
def train_step(model: nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
    model.train()

    # setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # loop through dataloader batches
    for batch, (X, y) in enumerate(dataloader):
        # send the data into the target device
        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()   # get the single integer

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # calculate accuracy metric
        pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (pred_class == y).sum().item() / len(y_pred)
    
    # adjust metrics to get average loss and accuracy per batch
    # -> aberage loss and accuracy per epoch across all batches
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc


# testing steps
def test_step(model:nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              device: torch.device):
    test_loss, test_acc = 0, 0

    model.eval()
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            test_pred = model(X)

            loss = loss_fn(test_pred, y)
            test_loss += loss.item()

            test_label = test_pred.argmax(dim=1)
            test_acc += (test_label == y).sum().item() / len(test_label)

    # adjust metrics to get average per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)

    return test_loss, test_acc


# train function -> put training step and testing step together
def train(model: nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: nn.Module,
          epochs: int,
          device: torch.device):
    # create empty results dictionary
    results = {"train_loss" : [],
               "train_acc": [],
               "test_loss" : [],
               "test_acc": []}
    
    # loop through train and test loop
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )
        # update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    
    return results

Writing go_modular/engine.py


In [12]:
from go_modular import engine
from torch import nn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

engine.train(model_0, train_dataloader, test_dataloader, optimizer, loss_fn, 5, "cpu")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1019 | train_acc: 0.3594 | test_loss: 1.1034 | test_acc: 0.2973
Epoch: 2 | train_loss: 1.1026 | train_acc: 0.2930 | test_loss: 1.1247 | test_acc: 0.2576
Epoch: 3 | train_loss: 1.0891 | train_acc: 0.4570 | test_loss: 1.1166 | test_acc: 0.3598
Epoch: 4 | train_loss: 1.0800 | train_acc: 0.4258 | test_loss: 1.1093 | test_acc: 0.3598
Epoch: 5 | train_loss: 1.1175 | train_acc: 0.3047 | test_loss: 1.1240 | test_acc: 0.3002


{'train_loss': [1.1018784642219543,
  1.1025919318199158,
  1.089074045419693,
  1.0799826383590698,
  1.1175467371940613],
 'train_acc': [0.359375, 0.29296875, 0.45703125, 0.42578125, 0.3046875],
 'test_loss': [1.1034072240193684,
  1.1246881087621052,
  1.1165741682052612,
  1.109251340230306,
  1.1240050792694092],
 'test_acc': [0.29734848484848486,
  0.25757575757575757,
  0.35984848484848486,
  0.35984848484848486,
  0.300189393939394]}

### 5. Create a file called `utils.py` with utility functions

=> save the model

In [13]:
%%writefile go_modular/utils.py
""" cintains various utility functions for PyTorvh model training """

import torch
from torch import nn
from pathlib import Path

def save_model(model: nn.Module, target_dir: str, model_name: str):
    """ save a model yo a target directory """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # create the model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    print("[INFO] Saving model to:", model_save_path)
    torch.save(obj=model.state_dict(), f=model_save_path)


Writing go_modular/utils.py


### 6. Train, evaluate and save model -> `train.py`

=> create a file called `train.py` to leverage all other code scripts to train a PyTorch model`

In [14]:
%%writefile go_modular/train.py
""" Trains a PyTorch image classification model """

import os
import torch
from torch import nn
from torchvision import transforms
from timeit import default_timer as timer 

import data_setup, engine, model_builder, utils

# Setup hyperparameters
NUM_EPOCHS = 5 
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
DEVICE = "cpu"

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create dataloader and get class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# creata a model
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(class_names))

# set up loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# train the model and set the timer
start_time = timer()
engine.train(model,
             train_dataloader,
             test_dataloader,
             optimizer,
             loss_fn,
             NUM_EPOCHS,
             DEVICE)
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# save the model to file
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_go_modular_script_tinyvgg_model.pth")

Writing go_modular/train.py


In [15]:
!python go_modular/train.py

100%|█████████████████████████████████████████████| 5/5 [00:06<00:00,  1.25s/it]
[INFO] Total training time: 6.278 seconds
[INFO] Saving model to: models/05_go_modular_script_tinyvgg_model.pth
